In [1]:
%pip install pandas


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd

In [47]:
def json_to_matrix_df(data, label: str = ""):
    # Convert each metric's dictionary to a pandas Series
    series_dict = {}
    for metric, stats in data.items():
        if metric == "chunkCount":
            # store as int data type
            series_dict[metric] = pd.Series(stats).astype("int32")
        elif metric in ["timeToFirstChunk", "totalTime"]:
            series_dict[metric] = (pd.Series(stats) / 1_000).round(2)
        else:
            series_dict[metric] = pd.Series(stats).round(2)
    
    # Combine all series into a DataFrame
    df = pd.DataFrame(series_dict).T

    # add label to df
    if label:
        df.index.name = label
    
    return df

In [50]:
import json

def compare_and_display_data(source1: str, source2: str):
    with open(source1) as file:
        d1 = json.loads(file.read())

    with open(source2) as file:
        d2 = json.loads(file.read())
    
    keys = set(d1.keys()).union(set(d2.keys()))
    print(keys)
    for key in keys:
        print("-" * 5, key, "-" * 5)

        if key in d1:
            display(json_to_matrix_df(d1[key], label=source1))

        if key in d2:
            display(json_to_matrix_df(d2[key], label=source2))

### Ollama vs vLLM

1. **concurrent, cold-start:** 10 concurrent requests when container is _NOT_ up 
2. **concurrent, warm:** 10 concurrent requests after container is already up **(warm)**
3.  **sequential, cold-start:** 10 _sequential_ requests, when the container is NOT up

In [51]:
compare_and_display_data("./ollama/stats.json", "./vllm/stats.json")

{'concurrent-10-same-prompt-warm', 'concurrent-10-sp-OLLAMA_NUM_PARALLEL-10', 'concurrent-10-same-prompt-cold-start', 'sequential-same-prompt'}
----- concurrent-10-same-prompt-warm -----


,min,max,average,p50,p75,p95,p99,p100
./ollama/stats.json,,,,,,,,
timeToFirstChunk,2.45,16.46,7.56,8.47,9.49,15.18,16.21,16.46
totalTime,7.82,21.56,14.30,14.99,18.00,21.08,21.46,21.56
chunkCount,251.00,425.00,323.00,323.00,346.00,392.00,418.00,425.00
averageChunksPerSecond,13.91,34.97,25.00,21.90,33.51,34.75,34.93,34.97


,min,max,average,p50,p75,p95,p99,p100
./vllm/stats.json,,,,,,,,
timeToFirstChunk,0.57,0.62,0.59,0.59,0.59,0.61,0.62,0.62
totalTime,8.44,13.02,10.83,10.79,11.46,12.89,12.99,13.02
chunkCount,267.00,424.00,348.00,346.00,369.00,419.00,423.00,424.00
averageChunksPerSecond,31.62,32.57,32.09,32.07,32.23,32.54,32.57,32.57


----- concurrent-10-sp-OLLAMA_NUM_PARALLEL-10 -----


,min,max,average,p50,p75,p95,p99,p100
./ollama/stats.json,,,,,,,,
timeToFirstChunk,0.49,0.54,0.52,0.53,0.53,0.54,0.54,0.54
totalTime,12.35,15.57,13.79,14.08,14.16,15.27,15.51,15.57
chunkCount,301.00,440.00,349.00,353.00,355.00,420.00,436.00,440.00
averageChunksPerSecond,24.22,28.26,25.25,25.05,25.13,27.53,28.12,28.26


...

,min,max,average,p50,p75,p95,p99,p100
./ollama/stats.json,,,,,,,,
timeToFirstChunk,10.23,25.44,16.49,18.04,18.53,25.38,25.43,25.44
totalTime,17.76,32.60,23.89,25.33,25.86,32.01,32.48,32.60
chunkCount,303.00,444.00,361.00,356.00,377.00,419.00,439.00,444.00
averageChunksPerSecond,10.97,21.00,15.87,13.76,20.42,20.84,20.97,21.00


,min,max,average,p50,p75,p95,p99,p100
./vllm/stats.json,,,,,,,,
timeToFirstChunk,66.34,66.65,66.62,66.65,66.65,66.65,66.65,66.65
totalTime,74.20,79.33,76.77,76.85,77.47,78.67,79.20,79.33
chunkCount,257.00,435.00,345.00,348.00,370.00,410.00,430.00,435.00
averageChunksPerSecond,3.46,5.48,4.48,4.53,4.79,5.22,5.43,5.48


----- sequential-same-prompt -----


,min,max,average,p50,p75,p95,p99,p100
./ollama/stats.json,,,,,,,,
timeToFirstChunk,0.47,10.20,1.47,0.50,0.52,5.84,9.33,10.20
totalTime,4.51,14.49,6.12,5.29,5.49,10.56,13.71,14.49
chunkCount,313.00,404.00,360.00,363.00,375.00,398.00,402.00,404.00
averageChunksPerSecond,23.25,70.90,65.33,69.92,70.20,70.72,70.86,70.90


,min,max,average,p50,p75,p95,p99,p100
./vllm/stats.json,,,,,,,,
timeToFirstChunk,0.45,63.89,6.84,0.50,0.54,35.4,58.19,63.89
totalTime,8.47,74.39,16.39,10.15,11.08,46.0,68.71,74.39
chunkCount,291.00,393.00,348.00,352.00,378.00,391.0,392.00,393.00
averageChunksPerSecond,5.14,34.80,31.69,34.68,34.70,34.8,34.80,34.80


## Ollama vs vLLM: batched concurrent requests: 1, 2, ... 10

In [6]:
import json
import pandas as pd
from typing import Dict, Any
from pathlib import Path

def batched_json_to_df(file_path: str, field: str) -> pd.DataFrame:
    """
    Convert a nested JSON file containing performance metrics into a 2D pandas DataFrame.
    
    Parameters:
    -----------
    file_path : str
        Path to the JSON file containing nested performance metrics
    field : str
        The metric field to extract (e.g., "timeToFirstChunk", "totalTime")
        
    Returns:
    --------
    pd.DataFrame
        A DataFrame where:
        - Index: Number of concurrent requests (1, 2, 3, ...)
        - Columns: Statistics (min, max, average, p50, ...)
        - Values: The corresponding metric values
        
    Example:
    --------
    >>> df = json_to_df("metrics.json", "timeToFirstChunk")
    >>> print(df)
              min       max   average       p50       p75       p95       p99      p100
    1  542.062209 542.06221 542.06221 542.06221 542.06221 542.06221 542.06221 542.06221
    2  369.981792 644.45421 507.21800 507.21800 575.83610 630.73059 641.70948 644.45421
    """
    
    # Validate and read JSON file
    try:
        with open(file_path, 'r') as f:
            data: Dict[str, Any] = json.load(f)
    except FileNotFoundError:
        raise FileNotFoundError(f"Could not find file: {file_path}")
    except json.JSONDecodeError:
        raise ValueError(f"Invalid JSON format in file: {file_path}")
    
    # Validate field exists
    if not any(field in entry for entry in data.values()):
        raise ValueError(f"Field '{field}' not found in data")
    
    # Define stat columns in desired order
    stat_columns = ['min', 'max', 'average', 'p50', 'p75', 'p95', 'p99', 'p100']
    
    # Extract data into dictionary for DataFrame
    df_data = {}
    for req_num, metrics in data.items():
        if field in metrics:
            df_data[int(req_num)] = {
                stat: metrics[field][stat]
                for stat in stat_columns
                if stat in metrics[field]
            }
    
    # Create DataFrame and sort index
    df = pd.DataFrame.from_dict(df_data, orient='index')
    df.sort_index(inplace=True)
    
    return df

## Ollama vs vLLM: batched concurrent requests: 1, 2, ... 10

In [7]:
print("----- time to first chunk (seconds) -----")
print("ollma:")
display(batched_json_to_df("./ollama/batched-concurrent-1-10-same-prompt-warm/stats.json", "timeToFirstChunk") / 1_000)
print("vLLM:")
display(batched_json_to_df("./vllm/batched-concurrent-1-10-same-prompt-warm/stats.json", "timeToFirstChunk") / 1_000)

----- time to first chunk (seconds) -----
ollma:


,min,max,average,p50,p75,p95,p99,p100
1,0.635483,0.635483,0.635483,0.635483,0.635483,0.635483,0.635483,0.635483
2,0.343561,0.499343,0.421452,0.421452,0.460398,0.491554,0.497785,0.499343
3,0.350625,0.504209,0.411276,0.378995,0.441602,0.491688,0.501705,0.504209
4,0.431060,0.505022,0.450045,0.432048,0.450763,0.494170,0.502851,0.505022
5,0.355785,7.517658,1.806057,0.386096,0.386708,6.091468,7.232420,7.517658
6,0.350514,8.976550,2.850625,0.499497,4.837467,8.303116,8.841863,8.976550
7,0.344395,8.467391,3.587836,0.517500,7.436944,8.177896,8.409492,8.467391
8,0.349951,8.905904,4.152758,3.745244,7.532830,8.765310,8.877785,8.905904
9,0.346472,15.153808,5.659753,8.038291,8.456670,12.853918,14.693830,15.153808
10,0.354370,15.297733,6.206700,7.267466,8.127731,14.974463,15.233079,15.297733


vLLM:


,min,max,average,p50,p75,p95,p99,p100
1,0.542062,0.542062,0.542062,0.542062,0.542062,0.542062,0.542062,0.542062
2,0.369982,0.644454,0.507218,0.507218,0.575836,0.630731,0.641709,0.644454
3,0.359018,0.489078,0.407376,0.374030,0.431554,0.477573,0.486777,0.489078
4,0.354277,0.480627,0.391681,0.365909,0.395155,0.463533,0.477208,0.480627
5,0.361354,0.495664,0.422386,0.382990,0.495014,0.495534,0.495638,0.495664
6,0.355050,0.463725,0.400752,0.374868,0.440305,0.463296,0.463639,0.463725
7,0.363855,0.501545,0.428470,0.379763,0.499517,0.501363,0.501509,0.501545
8,0.351668,0.465507,0.401690,0.386588,0.406433,0.465042,0.465414,0.465507
9,0.361868,0.482690,0.412114,0.384279,0.473506,0.479588,0.482070,0.482690
10,0.364537,0.515742,0.415856,0.394980,0.396548,0.514997,0.515593,0.515742


In [8]:
print("----- tokens per second -----")
print("ollma:")
display(batched_json_to_df("./ollama/batched-concurrent-1-10-same-prompt-warm/stats.json", "averageChunksPerSecond"))
print("vLLM:")
display(batched_json_to_df("./vllm/batched-concurrent-1-10-same-prompt-warm/stats.json", "averageChunksPerSecond"))

----- tokens per second -----
ollma:


,min,max,average,p50,p75,p95,p99,p100
1,63.699260,63.699260,63.699260,63.699260,63.699260,63.699260,63.699260,63.699260
2,56.466506,57.223739,56.845122,56.845122,57.034431,57.185877,57.216167,57.223739
3,46.263104,49.889997,48.441111,49.170233,49.530115,49.818021,49.875602,49.889997
4,43.625138,48.814938,45.466923,44.713808,45.932112,48.238373,48.699625,48.814938
5,29.539593,45.959037,41.115805,43.730573,44.030917,45.573413,45.881912,45.959037
6,22.424792,44.574557,37.472731,43.114258,44.155058,44.527403,44.565126,44.574557
7,21.534279,44.524877,34.585958,42.978923,43.664167,44.465176,44.512937,44.524877
8,20.816513,44.443130,33.445156,34.047287,44.002529,44.372355,44.428975,44.443130
9,13.151860,44.479134,30.753927,23.995550,44.220978,44.420851,44.467477,44.479134
10,17.413757,44.130455,30.245062,23.932957,43.767398,44.024654,44.109295,44.130455


vLLM:


,min,max,average,p50,p75,p95,p99,p100
1,34.703613,34.703613,34.703613,34.703613,34.703613,34.703613,34.703613,34.703613
2,33.822076,34.388686,34.105381,34.105381,34.247034,34.360356,34.383020,34.388686
3,33.908592,34.119917,34.027472,34.053906,34.086912,34.113316,34.118597,34.119917
4,33.005715,34.307448,33.728006,33.799430,34.045246,34.255008,34.296960,34.307448
5,33.026098,34.023151,33.392229,33.219605,33.476973,33.913915,34.001304,34.023151
6,32.809783,33.800736,33.373115,33.410089,33.562994,33.744727,33.789534,33.800736
7,32.744450,33.539080,33.099907,33.002321,33.277219,33.525108,33.536286,33.539080
8,31.978840,33.388816,32.953494,33.113384,33.225179,33.358438,33.382741,33.388816
9,32.263658,33.304383,32.749525,32.607271,33.046268,33.204950,33.284496,33.304383
10,32.287576,32.787016,32.543437,32.533002,32.606683,32.760491,32.781711,32.787016
